In [1]:
import os
import shutil
from PIL import Image
import numpy as np
import scipy as sp
from scipy.spatial import distance
import time
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

# Device configuration
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
#NVIDIA GT 630M - 2.1 <= 3.0 !!! cuDNN error: CUDNN_STATUS_ARCH_MISMATCH
device = torch.device('cpu')
device

device(type='cpu')

In [59]:
dir_ = "data"
market_path_train = "/Market-1501-v15.09.15/bounding_box_train"
market_path_test = "/Market-1501-v15.09.15/bounding_box_test"
dist_market_path_train = "/trainClasses"
dist_market_path_test = "/testClasses"
dist_market_path_smallTest = "/smallTest"
files_train = os.listdir(dir_ + market_path_train)
files_test = os.listdir(dir_ + market_path_test)

In [60]:
def load_dataset(path, batch_size=1, shuffle=False):
    data_path = dir_ + path
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=shuffle
    )
    return train_loader

In [25]:
train_loader = load_dataset(dist_market_path_train, batch_size=1000, shuffle=True)

In [61]:
test_loader = load_dataset(dist_market_path_smallTest, batch_size=1, shuffle=False)

## Train

In [62]:
#inplace = 3 - кол-во входных каналов (RGB)

class CNN_ReID(nn.Module):
    def __init__(self, inplanes=3, planes1=8, planes2=16, planes3=32, stride=1):
        super(CNN_ReID, self).__init__()

        self.layer1 = nn.Sequential(nn.Conv2d(inplanes, planes1, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.layer2 = nn.Sequential(nn.Conv2d(planes1, planes2, kernel_size=3, stride=1, padding=1), 
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(planes2, planes3, kernel_size=3, stride=1, padding=1), 
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)) 
        #self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(16 * 8 * planes3, 128) # 16*8*32 = 4096
        self.fc2 = nn.Linear(128, 751) # 16*8*32 = 4096
    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        #out = self.drop_out(out)
        out1 = self.fc1(out)
        out2 = self.fc2(out1)
        return out2, out1

In [8]:
def trainModel(num_epochs=10, weight_decay=0.001, momentum=0.9, learning_rate=0.01, name="model_0.ckpt"):
    model = CNN_ReID().to(device)
    criterion = nn.CrossEntropyLoss() # CrossEntropyLoss() объединяет и SoftMax, и кросс-энтропийную функцию потерь
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

    total_step = len(train_loader)
    loss_list = []
    acc_list = []

    print("wd =",weight_decay, "momentum =", momentum, "lr = ", learning_rate)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device) 
            labels = labels.to(device)

            # Прямой запуск
            outputs2, outputs1 = model(images)
            loss = criterion(outputs2, labels)
            loss_list.append(loss.item())

            # Обратное распространение и оптимизатор
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Отслеживание точности
            total = labels.size(0)
            _, predicted = torch.max(outputs2.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)

            if (i + 1) % (total_step-1) == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.8f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total)))
                
    print("LOSS = ", loss.item(), "wd =", weight_decay, "momentum =", momentum, "lr = ", learning_rate)
    
    # Save the model checkpoint
    torch.save(model.state_dict(), name)

In [9]:
start_time = time.time()
trainModel(num_epochs=30, weight_decay=0.001, momentum=0.9, learning_rate=0.05, name="model_task5_0.ckpt")
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.05
Epoch [1/30], Step [12/13], Loss: 6.6158, Accuracy: 0.00100000%
Epoch [2/30], Step [12/13], Loss: 6.5811, Accuracy: 0.00200000%
Epoch [3/30], Step [12/13], Loss: 6.5302, Accuracy: 0.00600000%
Epoch [4/30], Step [12/13], Loss: 6.4173, Accuracy: 0.00900000%
Epoch [5/30], Step [12/13], Loss: 6.2056, Accuracy: 0.02200000%
Epoch [6/30], Step [12/13], Loss: 5.8577, Accuracy: 0.03700000%
Epoch [7/30], Step [12/13], Loss: 5.5557, Accuracy: 0.04600000%
Epoch [8/30], Step [12/13], Loss: 5.1503, Accuracy: 0.08000000%
Epoch [9/30], Step [12/13], Loss: 4.7737, Accuracy: 0.10700000%
Epoch [10/30], Step [12/13], Loss: 4.5008, Accuracy: 0.12900000%
Epoch [11/30], Step [12/13], Loss: 4.2545, Accuracy: 0.16000000%
Epoch [12/30], Step [12/13], Loss: 4.2873, Accuracy: 0.19500000%
Epoch [13/30], Step [12/13], Loss: 3.4205, Accuracy: 0.29800000%
Epoch [14/30], Step [12/13], Loss: 2.9872, Accuracy: 0.36400000%
Epoch [15/30], Step [12/13], Loss: 2.5919, Accuracy: 0.4280000

In [10]:
start_time = time.time()
trainModel(num_epochs=30, weight_decay=0.001, momentum=0.9, learning_rate=0.05, name="model_task5_1.ckpt")
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.05
Epoch [1/30], Step [12/13], Loss: 6.6177, Accuracy: 0.00300000%
Epoch [2/30], Step [12/13], Loss: 6.6062, Accuracy: 0.00500000%
Epoch [3/30], Step [12/13], Loss: 6.5671, Accuracy: 0.00500000%
Epoch [4/30], Step [12/13], Loss: 6.5026, Accuracy: 0.00300000%
Epoch [5/30], Step [12/13], Loss: 6.4812, Accuracy: 0.00500000%
Epoch [6/30], Step [12/13], Loss: 6.4120, Accuracy: 0.00800000%
Epoch [7/30], Step [12/13], Loss: 6.2185, Accuracy: 0.01800000%
Epoch [8/30], Step [12/13], Loss: 5.8261, Accuracy: 0.03600000%
Epoch [9/30], Step [12/13], Loss: 5.4581, Accuracy: 0.03900000%
Epoch [10/30], Step [12/13], Loss: 5.1307, Accuracy: 0.07200000%
Epoch [11/30], Step [12/13], Loss: 4.7584, Accuracy: 0.10700000%
Epoch [12/30], Step [12/13], Loss: 4.2373, Accuracy: 0.15200000%
Epoch [13/30], Step [12/13], Loss: 3.9584, Accuracy: 0.21200000%
Epoch [14/30], Step [12/13], Loss: 3.3969, Accuracy: 0.30400000%
Epoch [15/30], Step [12/13], Loss: 2.9130, Accuracy: 0.3750000

## Test

In [63]:
model = CNN_ReID()
model.load_state_dict(torch.load("model_task5_0.ckpt"))
model.eval()

CNN_ReID(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=751, bias=True)
)

In [64]:
len(test_loader)

1926

In [65]:
test_features = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs2, outputs1 = model(images)
        test_features.append((str(labels.tolist()[0]), outputs1, images[0].numpy()))

In [66]:
def find_distance_one(test_features, index):
    list_dist = []
    for i in range(len(test_features)):
        if i != index:
            dist_ = distance.cosine(test_features[index][1], test_features[i][1])
            list_dist.append((test_features[i][0], dist_, test_features[i][2]))
            list_dist.sort(key=lambda tup: tup[1])
    result = (test_features[index][0], list_dist)
    return result

In [67]:
def find_distance_all(test_features):
    result = []
    for i in tqdm_notebook(range(len(test_features))):
        distances_for_index = find_distance_one(test_features=test_features, index=i)
        result.append(distances_for_index)
    return result

In [80]:
def mAP(rangs, path_="data/testClasses/"):
    #{ { str_path, { {str_path_r_1, p_r_1}, {str_path_r_2, p_r_2}, .. {str_path_r_n, p_r_1} } }, ... }
 
    len_rangs = len(rangs) #751
    print(len_rangs)
    sum_mAP = 0

    for rang in rangs:
        person = str(rang[0])
        len_person = len(os.listdir(path_ + str("{:0>4s}".format(person)) + '/')) - 1
        sum_P = 0
        sum_true = 0
        i = 0
        for rang_r in rang[1]:
            i += 1
            if rang_r[0] == person:
                sum_true += 1
                sum_P += sum_true/i
            #print("Person: ", person, rang_r[0].split('_')[0], sum_true/i)
        #print("Person:", person, "len_person:", str(len_person), "AP =", sum_P/len_person)
        sum_mAP += sum_P/len_person
    return sum_mAP/len_rangs

In [72]:
test_features[58][0]

'0'

In [73]:
rangs = find_distance_all(test_features=test_features)

In [54]:
rangs[843][0]

'49'

In [81]:
mAP(rangs=rangs, path_="data/smallTest/")

1926


0.27224515098611113